# A solution to Watson's contradictions
### From Kaggle's recurring challenge "Contradictory, My Dear Watson"


This is a preliminary model to a Natural Language Inferencing (NLI) problem, which takes two sentences and decides if the first entails, contradicts, or is unrelated to the second.

See the challenge on Kaggle [here](https://www.kaggle.com/c/contradictory-my-dear-watson/overview).

In [5]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

Matplotlib is building the font cache; this may take a moment.


Set up the TPU.

In [6]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  strategy = tf.distribute.get_strategy()
  print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1
